In [5]:
import csv
import os

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import jieba

In [6]:
# 读取csv文件中的数据
keywords_filepath = os.path.join('test_output','keywords.csv')
data = pd.read_csv(keywords_filepath,encoding='utf-8')

# 提取标签和目标文本
labels_true = data.iloc[:, 1]
documents = data.iloc[:, 2]
documents = [" ".join(jieba.cut_for_search(doc)) for doc in documents]

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\zaz\AppData\Local\Temp\jieba.cache
Loading model cost 0.784 seconds.
Prefix dict has been built successfully.


In [7]:
# 使用TF-IDF进行特征提取
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(documents)

In [11]:
X

<795x1602 sparse matrix of type '<class 'numpy.float64'>'
	with 4923 stored elements in Compressed Sparse Row format>

In [8]:
# 使用KMeans进行聚类
true_k = 4 # 设定聚类数量，可以根据需要调整
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=1, n_init=1)
model.fit(X)

KMeans(max_iter=1, n_clusters=4, n_init=1)

In [17]:
X[1,1]

0.0

In [9]:
# 输出聚类结果
print("每个类的关键词")
# 这里的质心则代表的每个词语的重要程度
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names_out()

每个类的关键词


In [15]:
# 这里的意思就是拿到聚类中心，聚类则是把重要关键字相同的书籍聚集在一起，
model.cluster_centers_.shape
order_centroids.shape

(4, 1602)

In [10]:
cluster_result = os.path.join('test_output','keywords.csv')
with open(cluster_result, 'w', newline='') as f:
    writer = csv.writer(f)
    for i in range(true_k):
        print(f"Cluster {i}:")
        out = ""
        for ind in order_centroids[i, :10]:#这里可以选择输出关键词的数量
            out = out + terms[ind]+ "、"
            print(f' {terms[ind]}')
        writer.writerow([i, out])
f.close()

# 为每个选题分配标签
labels = model.labels_
with open('D:\\工作\\聚类：每本书的标签.csv', 'w', newline='') as f:
    for i, (true_label, doc, cluster_label) in enumerate(zip(labels_true, documents, labels)):
        writer = csv.writer(f)
        writer.writerow([i, true_label, cluster_label])
        print(f"Document {i} (True label: {true_label}): Cluster {cluster_label}")
f.close()

with open('D:\\工作\\聚类：每个类下的书.csv', 'w', newline='') as f:
    for i in range(true_k):
        cluster_documents = [labels_true[idx] for idx in range(len(labels)) if labels[idx] == i]
        writer = csv.writer(f)
        writer.writerow([i, cluster_documents])
        print(f"Cluster {i} documents:")
        print(cluster_documents)
        print()
f.close()

Cluster 0:
 生命
 人生
 意义
 自然
 探索
 力量
 感悟
 敬畏
 珍视
 生活
Cluster 1:
 探索
 力量
 传承
 智慧
 文化
 科学
 精神
 艺术
 未知
 启蒙
Cluster 2:
 生活
 热爱
 热爱生活
 快乐
 自我
 乐观
 珍视
 自由
 当下
 心灵
Cluster 3:
 自我
 成长
 积极
 独立
 女性
 认知
 和解
 自己
 希望
 健康


FileNotFoundError: [Errno 2] No such file or directory: 'D:\\工作\\聚类：每本书的标签.csv'